# Setup

In [ ]:
import os
import zipfile

%cd /content

! git clone --recurse-submodules https://github.com/jagrut-thakare/ghost-2.0
%cd ghost-2.0
! git submodule init
! git submodule update
os.makedirs("aligner_checkpoints", exist_ok=True)
os.makedirs("blender_checkpoints", exist_ok=True)
os.makedirs("src/losses/gaze_models", exist_ok=True)
os.makedirs("weights", exist_ok=True)

! wget -O aligner_checkpoints/aligner_1020_gaze_final.ckpt https://github.com/ai-forever/ghost-2.0/releases/download/aligner/aligner_1020_gaze_final.ckpt
! wget -O blender_checkpoints/blender_lama.ckpt https://github.com/ai-forever/ghost-2.0/releases/download/aligner/blender_lama.ckpt
! wget -O weights/backbone50_1.pth https://github.com/ai-forever/ghost-2.0/releases/download/aligner/backbone50_1.pth
! wget -O weights/vgg19-d01eb7cb.pth https://github.com/ai-forever/ghost-2.0/releases/download/aligner/vgg19-d01eb7cb.pth
! wget -O weights/segformer_B5_ce.onnx https://github.com/ai-forever/ghost-2.0/releases/download/aligner/segformer_B5_ce.onnx
! wget -O gaze_models.zip https://github.com/ai-forever/ghost-2.0/releases/download/aligner/gaze_models.zip

with zipfile.ZipFile("gaze_models.zip", 'r') as zip_ref:
    zip_ref.extractall("src/losses/")

os.remove("gaze_models.zip")
print("✅ All models downloaded and extracted successfully.")

! mkdir repos

%cd repos

! git clone --recurse-submodules https://github.com/yfeng95/DECA
%cd DECA
! git submodule init
! git submodule update
%cd ..

! git clone --recurse-submodules https://github.com/jagrut-thakare/emoca
%cd emoca

! git submodule init
! git submodule update

os.makedirs("gdl_apps/EmotionRecognition", exist_ok=True)
os.makedirs("assets/EmotionRecognition/image_based_networks", exist_ok=True)

!wget -O ResNet50.zip https://github.com/anastasia-yaschenko/emoca/releases/download/resnet/ResNet50.zip

with zipfile.ZipFile("ResNet50.zip", 'r') as zip_ref:
    zip_ref.extractall("gdl_apps/EmotionRecognition")
    zip_ref.extractall("assets/EmotionRecognition/image_based_networks")

os.remove("ResNet50.zip")

print("✅ ResNet checkpoints downloaded, unpacked, and ZIP removed.")
%cd ..

! git clone --recurse-submodules https://github.com/anastasia-yaschenko/BlazeFace_PyTorch

%cd BlazeFace_PyTorch
! git submodule init
! git submodule update
%cd ..

! git clone --recurse-submodules https://github.com/chroneus/stylematte

%cd stylematte

! git submodule init
! git submodule update

os.makedirs("stylematte/checkpoints", exist_ok=True)
os.makedirs("stylegan3", exist_ok=True)

!wget -O stylematte/checkpoints/stylematte_pure.pth https://github.com/chroneus/stylematte/releases/download/weights/stylematte_pure.pth
!wget -O stylematte/checkpoints/stylematte_synth.pth https://github.com/chroneus/stylematte/releases/download/weights/stylematte_synth.pth
!wget -O stylegan3/animals.pkl https://github.com/chroneus/stylematte/releases/download/weights/animals.pkl
!wget -O stylegan3/humans.pkl https://github.com/chroneus/stylematte/releases/download/weights/humans.pkl
%cd ..

%cd ..

In [ ]:
! conda config --add channels conda-forge
! conda config --set channel_priority strict
! conda create -n ghost python=3.10

In [ ]:
! pip install -r requirements.txt

In [ ]:
! conda install -c pytorch3d-nightly pytorch3d
! pip install face-alignment==1.3.5 facenet-pytorch==2.5.2 -U git+https://github.com/facebookresearch/fvcore

In [ ]:
! conda install scikit-image -y

In [ ]:
! conda install -c conda-forge imgaug -y

In [ ]:
! conda install numpy==1.23.5 -y

In [ ]:
! conda install tensorboardX tensorboard -y

# DECA Model

In [ ]:
import gdown

file_id = "1rp8kdyLPvErw2dTmqtjISRVvQLj6Yzje"
destination = "/content/ghost-2.0/repos/DECA/data/deca_model.tar"

gdown.download(id=file_id, output=destination, quiet=False)

In [ ]:
import os
from google.colab import userdata

# Create data directory
os.makedirs("data", exist_ok=True)

# Set FLAME credentials (assumed to be already set)
username = userdata.get('DECA')
password = userdata.get('DECA_Password')

# Check if credentials are provided
if not username or not password:
    raise ValueError("Environment variables  must be set.")

# Encode username and password for URL
import urllib.parse
username_enc = urllib.parse.quote_plus(username)
password_enc = urllib.parse.quote_plus(password)

# Create the download command
download_url = f"https://download.is.tue.mpg.de/download.php?domain=flame&sfile=FLAME2020.zip&resume=1"
post_data = f"username={username_enc}&password={password_enc}"

# Use wget with POST data
!wget --post-data="{post_data}" "{download_url}" -O DECA.zip --no-check-certificate --continue

# Unzip and move model file
!unzip DECA.zip -d ./temp
!mv ./temp/* /content/ghost-2.0/repos/DECA/data/
!rm -rf ./temp/
!rm -rf DECA.zip

# EMOCA ResNet50

In [ ]:
import os
import gdown

# Create the directory
os.makedirs("/ps/scratch/rdanecek/FaceRecognition/", exist_ok=True)
# Google Drive file ID
file_id = "1gSUm_sZFOjcHij4wBptI3CEvmWQzVHoS"
output_path = "/ps/scratch/rdanecek/FaceRecognition/resnet50_ft_weight.pkl"

# Remove the file if it exists before downloading
if os.path.exists(output_path):
    os.remove(output_path)
    print(f"Removed existing file: {output_path}")

# Download the file
gdown.download(f"https://drive.google.com/uc?id={file_id}", output_path, quiet=False)
print(f"Downloaded file to: {output_path}")

# Inference

In [ ]:
import os
os.environ["MPLBACKEND"] = "agg"

In [5]:
! python inference.py --source ./examples/images/elon.jpg --target ./examples/images/hab.jpg --save_path result1.png --ckpt_a ./temp/aligner.ckpt --ckpt_b ./temp/blender.ckpt

/home/jagrut.thakare/anaconda3/envs/ghost/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
SWIN not found, will not be able to use SWIN models
Could not import EmoSwinModule. SWIN models will not be available.  Make sure you pull the repository with submodules to enable Swin.
Could not import EmoNetModule. EmoNet models will not be available.  Make sure you pull the repository with submodules to enable EmoNet.
Looking for checkpoint in 'repos/emoca/assets/EmotionRecognition/image_based_networks/ResNet50/checkpoints'
Found 1 checkpoints
 - repos/emoca/assets/EmotionRecognition/image_based_networks/ResNet50/checkpoints/deca-epoch=01-val_loss_total/dataloader_idx_0=1.27607644.ckpt
Selecting checkpoint 'repos/emoca/assets/EmotionRecognition/image_based_networks/ResNet50/checkpoints/deca-epoch=01-val_

# Training

In [ ]:
! conda run -n ghost python v2h5.py

In [ ]:
! conda run -n ghost pip install numpy==1.23 h5py

In [3]:
import os
os.environ["WANDB_MODE"] = "offline"

In [6]:
! python train_aligner.py --config ./configs/aligner.yaml

SWIN not found, will not be able to use SWIN models
Could not import EmoSwinModule. SWIN models will not be available.  Make sure you pull the repository with submodules to enable Swin.
Could not import EmoNetModule. EmoNet models will not be available.  Make sure you pull the repository with submodules to enable EmoNet.
Looking for checkpoint in 'repos/emoca/assets/EmotionRecognition/image_based_networks/ResNet50/checkpoints'
Found 1 checkpoints
 - repos/emoca/assets/EmotionRecognition/image_based_networks/ResNet50/checkpoints/deca-epoch=01-val_loss_total/dataloader_idx_0=1.27607644.ckpt
Selecting checkpoint 'repos/emoca/assets/EmotionRecognition/image_based_networks/ResNet50/checkpoints/deca-epoch=01-val_loss_total/dataloader_idx_0=1.27607644.ckpt'
/home/jagrut.thakare/anaconda3/envs/ghost/lib/python3.10/site-packages/pytorch_lightning/utilities/migration/migration.py:208: You have multiple `ModelCheckpoint` callback states in this checkpoint, but we found state keys that would end u

In [ ]:
! python train_blender.py --config ./configs/blender.yaml